In [1]:
import numpy as np
import pandas as pd
from click import style
from pandas import value_counts

import matplotlib.pyplot as plt


caminho_arquivo = r'C:\Users\Natan\OneDrive\Documents\Victor-projects\data projects\churn_telco\churn_telco\data\interim\dados_limpos_telco.xlsx'
df = pd.read_excel(caminho_arquivo)
df



,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,1,United States,California,Landers,92285,"34.341737, -116.539416",34.341737,-116.539416,Female,...,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No,0,45,5306,No opinion
7039,6840-RESVB,1,United States,California,Adelanto,92301,"34.667815, -117.536183",34.667815,-117.536183,Male,...,One year,Yes,Mailed check,84.80,1990.5,No,0,59,2140,No opinion
7040,2234-XADUH,1,United States,California,Amboy,92304,"34.559882, -115.637164",34.559882,-115.637164,Female,...,One year,Yes,Credit card (automatic),103.20,7362.9,No,0,71,5560,No opinion
7041,4801-JZAZL,1,United States,California,Angelus Oaks,92305,"34.1678, -116.86433",34.167800,-116.864330,Female,...,Month-to-month,Yes,Electronic check,29.60,346.45,No,0,59,2793,No opinion


In [2]:
# inicio do projeto:
# Somos analistas de dados e fomos contratados pela empresa americana Telco
# A principal missão é diminuir em 15% o nível de churn(Clientes com risco de cancelamento dos nossos produtos) em 3 meses com campanhas direcionadas
# Para isso vamos analisar o dataset disponibilizado pela companhia para ajudar a criar uma estratégia de ação para a empresa e diminuir a quandidade de churn
# Verificando a quantidade de linhas no dataframe
# Após uma verificação inicial é possivel notar muitos valores na coluna Churn Reason como nan.
# Présupondo que os valores nan seriam pessoas que não queriam dar feedback do serviço
# É possivel julgar a falta de feedback como insatisfação do serviço também
# Os valores NaN foram substituidos por "No opinion"
# Vamos remover eles criando um df filtrado para ter uma analise mais bem explicada

# printando valores sem "No opinion"
df_filtered = df[df["Churn Reason"] != "No opinion"].copy()

# Após remover valores "No opinion" vamos verificar quantas linhas temos no DF
quantity_filtered = df_filtered.shape[0]
print("Clientes com feedback definido: ", quantity_filtered)

#Calculando os 15% para a diminuição
goal_reduct = quantity_filtered * 0.15
print(f"Meta definida de redução: {goal_reduct:.0f} clientes")

# portanto, temos 1869 clientes com feedback definido. O nosso objetivo é diminuir a quantidade de churn em até 280 clientes


Clientes com feedback definido:  1869
Meta definida de redução: 280 clientes


In [21]:
print(df.columns)

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')


In [3]:
print(df_filtered["Churn Reason"].value_counts())


print("")
print("Motivo mais conhecido é o serviço de atendimento ao cliente(SAC)")

# Analisando mais profundamente a partir dos clientes que reclamaram do SAC
df_sac = df_filtered[df_filtered["Churn Reason"] == "Attitude of support person"].copy()

# analisando o score de churn na questão do atendimento ao cliente
print(df_sac["Churn Score"].dtype)

df_sac_churn_score = df_sac[df_sac["Churn Score"] >= 70]

print("Churn Score médio de quem reclamou do SAC:", df_sac["Churn Score"].mean())
print("Quantidade com score acima de 70:", df_sac_churn_score.shape[0])
print(df_sac_churn_score.describe())

# Podemos nos deparar com muitos problemas voltados ao SAC
# Em uma escala de 0 à 100 a média de churn é 82.04(uma média bem alta)
# Temos também 163 clientes que marcaram o score acima de 70
# Além disso, interpretando Tenure Months como tempo como clientes, temos uma média de 18.75 meses. O que é menos de 2 anos
# Em CLTV temos como valor total estimado do cliente. Outro dado muito importante, explicando quanto a empresa perde a cada cliente que cancela o serviço
print()
print("- Os clientes que reclamaram do SAC e tinham Churn Score alto")
print("- Clientes tinham em média menos de 2 anos de contrato")
print("- Pagavam em torno de $73 por mês")
print("- E representavam um alto valor estimado para a empresa (CLTV médio de $4164)")
print("- Score médio de 84, muitos deles estavam prontos para sair. Melhorar o SAC não é só questão de experiência do cliente, é uma jogada de salvar receita real.")


Churn Reason
Attitude of support person                   192
Competitor offered higher download speeds    189
Competitor offered more data                 162
Don't know                                   154
Competitor made better offer                 140
Attitude of service provider                 135
Competitor had better devices                130
Network reliability                          103
Product dissatisfaction                      102
Price too high                                98
Service dissatisfaction                       89
Lack of self-service on Website               88
Extra data charges                            57
Moved                                         53
Lack of affordable download/upload speed      44
Limited range of services                     44
Long distance charges                         44
Poor expertise of phone support               20
Poor expertise of online support              19
Deceased                                       6
Name: c

In [14]:
# comparando a média de score de churn de problemas com o SAC com a média de score de outros churn

# Segundo maior motivo de churn tem relação com qualidade do serviço, no caso, a concorrência oferece planos com maiores velocidades de download
churn_download_speed = df_filtered[df_filtered["Churn Reason"] == "Competitor offered higher download speeds"].copy()

df_download_speed_churn_score = churn_download_speed[churn_download_speed["Churn Score"] >= 70]
print("Churn médio sobre pessoas que reclamaram da velocidade da internet: ", df_download_speed_churn_score["Churn Score"].mean())
print("Quantidade com score acima de 70:", df_download_speed_churn_score.shape[0])

print("Resultados ainda piores do que as analises anteriores, e também já conseguimos ultrapassar os 15% de churn apenas com essas duas analises")
print("Além de investimentos na qualidade do atendimento, a empresa precisa realizar fortes ações de investimentos em melhorias de desempenho")
print("Seria eficiente trabalhar com melhorias e com o time de marketing para anunciar maiores velocidades posteriormente para seus clientes")
print()
print(df_download_speed_churn_score.describe())


Churn médio sobre pessoas que reclamaram da velocidade da internet:  85.3433734939759
Quantidade com score acima de 70: 166
Resultados ainda piores do que as analises anteriores, e também já conseguimos ultrapassar os 15% de churn apenas com essas duas analises
Além de investimentos na qualidade do atendimento, a empresa precisa realizar fortes ações de investimentos em melhorias de desempenho
Seria eficiente trabalhar com melhorias e com o time de marketing para anunciar maiores velocidades posteriormente para seus clientes

       Count      Zip Code    Latitude   Longitude  Tenure Months  \
count  166.0    166.000000  166.000000  166.000000     166.000000   
mean     1.0  93739.746988   36.326767 -119.877568      19.656627   
std      0.0   1737.454888    2.489080    2.363429      20.390346   
min      1.0  90013.000000   32.671170 -124.240051       1.000000   
25%      1.0  92241.500000   34.085686 -121.943632       3.000000   
50%      1.0  93815.500000   36.557613 -120.387793    

In [21]:
# O terceiro e ultimo motivo que vamos analisar será que a concorrência oferece maiores pacotes de dados

churn_data = df_filtered[df_filtered["Churn Reason"] == "Competitor offered more data"].copy()
churn_data_mean = churn_data[churn_data["Churn Score"] >= 70]
print("Média de pessoas que alegam maior quantidade de dados em empresas rivais: ", churn_data_mean["Churn Score"].mean())
print("Quantidade de pessoas que alegam maior quantidade de dados em outras empresas: ", churn_data_mean.shape[0])
print()
print(churn_data_mean.describe())

Média de pessoas que alegam maior quantidade de dados em empresas rivais:  84.6962962962963
Quantidade de pessoas que alegam maior quantidade de dados em outras empresas:  135

       Count      Zip Code    Latitude   Longitude  Tenure Months  \
count  135.0    135.000000  135.000000  135.000000     135.000000   
mean     1.0  93899.577778   36.780421 -120.200361      18.614815   
std      0.0   1807.165776    2.360327    2.198432      19.557029   
min      1.0  90019.000000   32.605012 -124.217378       1.000000   
25%      1.0  92590.000000   34.319741 -121.968077       3.500000   
50%      1.0  94403.000000   37.305622 -120.853288      10.000000   
75%      1.0  95464.500000   38.486720 -118.262006      30.000000   
max      1.0  96088.000000   41.950683 -114.192901      71.000000   

       Monthly Charges  Churn Value  Churn Score         CLTV  
count       135.000000        135.0   135.000000   135.000000  
mean         76.912222          1.0    84.696296  4215.051852  
std      